In [ ]:
import torch
import pandas as pd
from transformers import MarianMTModel, MarianTokenizer
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.meteor_score import meteor_score
from sacrebleu.metrics import TER
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import numpy as np

# Load CSV dataset
csv_file = "C:\\Users\\LIKIANU\\Downloads\\large_multilingual_translation_dataset.csv"
df = pd.read_csv(csv_file)

# Select first 10 rows as example input texts
input_texts = df['English'].head(10).tolist()

# Define language models for multiple translations
languages = {
    "French": "Helsinki-NLP/opus-mt-en-fr",
    "Spanish": "Helsinki-NLP/opus-mt-en-es",
    "German": "Helsinki-NLP/opus-mt-en-de",
    "Hindi": "Helsinki-NLP/opus-mt-en-hi",
    "Chinese": "Helsinki-NLP/opus-mt-en-zh"
}

# Store translations
translations = {lang: [] for lang in languages}

# Translate each sentence to multiple languages
for text in input_texts:
    for lang, model_name in languages.items():
        tokenizer = MarianTokenizer.from_pretrained(model_name)
        model = MarianMTModel.from_pretrained(model_name)
        
        inputs = tokenizer([text], return_tensors="pt", padding=True, truncation=True)
        translated = model.generate(**inputs)
        translated_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
        
        translations[lang].append(translated_text[0])

# Output translations
for lang, translated_texts in translations.items():
    print(f"\n{lang} Translations:")
    for text in translated_texts:
        print(text)

# Evaluate Model using BLEU Score
reference = [[word for word in df['French'].iloc[0].split()]]  # Reference translation
candidate = translations['French'][0].split()
bleu_score = sentence_bleu(reference, candidate)
print("BLEU Score:", bleu_score)

# Evaluate Model using METEOR Score
meteor_score_value = meteor_score(reference, candidate)
print("METEOR Score:", meteor_score_value)

# Evaluate Model using Translation Edit Rate (TER)
ter = TER()
reference_text = [df['French'].iloc[0]]
candidate_text = translations['French'][0]
ter_score = ter.corpus_score([candidate_text], [[ref] for ref in reference_text])
print("TER Score:", ter_score.score)

# Simulated Hyperparameter Tuning
learning_rates = [5e-5, 3e-4, 1e-3]
batch_sizes = [16, 32, 64]
best_lr, best_bs, best_score = None, None, float("-inf")
for lr in learning_rates:
    for bs in batch_sizes:
        score = np.random.rand()
        if score > best_score:
            best_score, best_lr, best_bs = score, lr, bs
print(f"Best Hyperparameters: Learning Rate={best_lr}, Batch Size={best_bs}, Score={best_score}")

# Cross-Validation (Simulated)
dataset = np.random.rand(100, 768)
kf = KFold(n_splits=5, shuffle=True, random_state=42)
for train_index, test_index in kf.split(dataset):
    train_data, test_data = dataset[train_index], dataset[test_index]
    predictions = np.random.randint(0, 2, size=len(test_data))

# Cross-Validation Accuracy
accuracy = accuracy_score(np.random.randint(0, 2, size=len(test_data)), predictions)
print("Cross-Validation Accuracy:", accuracy)